# RNN Model with crf

In [1]:
import rnn_dataset
import rnn_classifier
from crf import CRF
import torch
from data_utils import Vocabulary
from torch.utils.data import Dataset, DataLoader, random_split
from collections import Counter
import itertools
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from torch.nn.utils.rnn import pad_sequence
import pyconll


In [2]:
#define the hyperparameters
batch_size    = 16
lr            = 1e-2
device        = "cpu"
epochs        = 20
emb_size      = 64
hidden_size   = 64
nb_layers     = 2
drop_out      = 0.1

In [3]:
trainset     = rnn_dataset.MweRnnDataset("corpus/train.conllu",  isTrain = True)
testset      = rnn_dataset.MweRnnDataset("corpus/test.conllu")


token Vocab size 35693
token Vocab size 35693


In [4]:
for x, tag in trainset.get_loader(shuffle = True):
    print(list(trainset.tags_vocab.rev_lookup(int(t)) for t in tag.squeeze(0)))
    break
    

['B_V', 'B_P', 'B_N', 'B_P', 'B_V', 'B_D', 'B_N', 'B_A', 'B_PONCT', 'B_D', 'B_N', 'B_C', 'B_CL', 'B_V', 'B_V', 'B_ADV', 'B_V', 'B__', 'B_P', 'B_D', 'B_N', 'B_PONCT', '<pad>']


In [5]:
model = rnn_classifier.MweRNN(
    name         = "RNN",
    toks_vocab   = trainset.toks_vocab,
    tags_vocab   = trainset.tags_vocab, 
    emb_size     = emb_size, 
    hidden_size  = hidden_size, 
    drop_out     = drop_out)

In [ ]:
model.train_model(trainset,testset, epochs= epochs, lr=lr, batch_size = batch_size, split_train=0.8)

100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 77.71it/s]


Epoch 0 | Mean train loss  12.2721 |  Mean dev loss  6.5046 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 71.39it/s]


Epoch 1 | Mean train loss  4.1072 |  Mean dev loss  5.3728 



100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 77.06it/s]


...

100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 71.70it/s]


Epoch 5 | Mean train loss  1.0487 |  Mean dev loss  5.8031 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 50.35it/s]


Epoch 6 | Mean train loss  0.8808 |  Mean dev loss  6.0774 



  8%|███▍                                      | 47/581 [00:02<00:30, 17.35it/s]

In [ ]:
TP, FP, FN, average_precision, average_recall, average_f1_score, weighted_f1_score, weighted_recall, weighted_precision = model.evaluate()

In [64]:
num_tags = len(model.tags_vocab)
# print(num_tags)
TP = torch.zeros(num_tags)
FP = torch.zeros(num_tags)
FN = torch.zeros(num_tags)
class_counts = torch.zeros(num_tags)
with torch.no_grad():
            
    for X_toks, Y_golds in tqdm(testset.get_loader(batch_size = 500)):
    # Forward pass
        logprobs, masks = model.forward(X_toks)
        best_score, best_paths = model.crf(logprobs, masks) #viterbi
        #best_paths = pad_sequence(best_paths, padding_value= testset.tags_vocab["<pad>"]).T
        #print(best_paths.shape)
        # Mask out the padding positions
        
        for i in range(len(best_paths)):
            str = list(model.tags_vocab.rev_lookup(int(i))for i in gold if i!= model.padidx)
            path = best_paths[i]
            gold = torch.tensor([j for j in Y_golds[i] if j != model.padidx])
            for tag in path:
                TP[tag] += ((path == tag) & (gold == tag)).sum()
                FP[tag] += ((path == tag) & (gold != tag)).sum()
                FN[tag] += ((path != tag) & (gold == tag)).sum()
                class_counts[tag] += (gold == tag).sum()
                
            

100%|█████████████████████████████████████████████| 4/4 [00:04<00:00,  1.14s/it]


In [2]:
precision = TP / (TP + FP)
# avoid nan
nan_mask = torch.isnan(precision)
precision[nan_mask] = 0.

recall = TP / (TP + FN)
# avoid nan
nan_mask = torch.isnan(recall)
recall[nan_mask] = 0.

f1_score = 2 * (precision * recall) / (precision + recall)
# avoid nan
nan_mask = torch.isnan(f1_score)
f1_score[nan_mask] = 0.
# Calculate class weights
class_weights = class_counts / class_counts.sum()

NameError: name 'TP' is not defined

In [ ]:
for tag in range(num_tags):
    print(model.tags_vocab.rev_lookup(tag), precision[tag])

In [3]:
for tag in range(num_tags):
    print(model.tags_vocab.rev_lookup(tag), class_counts[tag])

NameError: name 'num_tags' is not defined

In [77]:
weighted_f1_score = (f1_score * class_weights)
weighted_recall = (recall * class_weights)
weighted_precision = (precision * class_weights)

In [15]:
print(sum(weighted_f1_score))
print(sum(weighted_recall))
print(sum(weighted_recall))

tensor(0.9184)
tensor(0.9386)
tensor(0.9386)


In [18]:
for tag in range(num_tags):
    print(model.tags_vocab.rev_lookup(tag), f1_score[tag])


<unk> tensor(0.)
<pad> tensor(0.)
B_CL tensor(0.9270)
I_V tensor(0.8889)
I_ADV tensor(0.2275)
B_V tensor(0.8962)
B_P tensor(0.9381)
B_A tensor(0.7716)
B_D tensor(0.9347)
B_N tensor(0.8759)
B_PONCT tensor(0.9828)
B_C tensor(0.9336)
B__ tensor(0.9997)
B_ADV tensor(0.3598)
I_N tensor(0.4318)
I_C tensor(0.4000)
I_CL tensor(0.9091)
I_D tensor(0.7143)
I_P tensor(0.3692)
I_PONCT tensor(0.8235)
I_A tensor(0.4545)
B_PREF tensor(0.8000)
B_I tensor(0.)
B_ET tensor(0.)
I_ET tensor(0.)
B_NC tensor(0.)
B_S tensor(0.)
B_X tensor(0.)


# LSTM Layer + CRF decoder

In [26]:
#define the hyperparameters
batch_size    = 16
window_size   =  6#left context and right context
lr            = 1e-2
device        = "cpu"
epochs        = 30
emb_size      = 64
hidden_size   = 64
drop_out      = 0.1

In [27]:
trainset     = rnn_dataset.MweRnnDataset("corpus/train.conllu",  isTrain = True)
testset      = rnn_dataset.MweRnnDataset("corpus/test.conllu")

token Vocab size 35693
token Vocab size 35693


In [28]:
model = rnn_classifier.MweRNN(
    name         = "LSTM",
    toks_vocab   = trainset.toks_vocab,
    tags_vocab   = trainset.tags_vocab, 
    emb_size     = emb_size, 
    hidden_size  = hidden_size, 
    drop_out     = 0.)

In [29]:
model.train_model(trainset,testset, epochs= epochs, lr=lr, batch_size = batch_size, split_train=0.8)

100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 88.52it/s]


Epoch 0 | Mean train loss  12.3865 |  Mean dev loss  6.3059 



100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 91.47it/s]


Epoch 1 | Mean train loss  4.2572 |  Mean dev loss  3.8725 



100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 78.67it/s]


...

Epoch 27 | Mean train loss  0.4597 |  Mean dev loss  0.4892 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 49.07it/s]


Epoch 28 | Mean train loss  0.3572 |  Mean dev loss  0.3485 



 46%|██████████████████▊                      | 266/581 [00:13<00:15, 19.98it/s]


KeyboardInterrupt: 

In [15]:
num_tags = len(model.tags_vocab)
# print(num_tags)
TP = torch.zeros(num_tags)
FP = torch.zeros(num_tags)
FN = torch.zeros(num_tags)
class_counts = torch.zeros(num_tags)
with torch.no_grad():
            
    for X_toks, Y_golds in tqdm(testset.get_loader(batch_size = 500)):
    # Forward pass
        logits, masks = model.forward(X_toks)
        best_score, best_paths = model.crf(logits, masks) #viterbi
        #best_paths = pad_sequence(best_paths, padding_value= testset.tags_vocab["<pad>"]).T
        #print(best_paths.shape)
        # Mask out the padding positions
        
        for i in range(len(best_paths)):
            str = list(model.tags_vocab.rev_lookup(int(i))for i in Y_golds[i] if i!= model.padidx)
            path = best_paths[i]
            gold = torch.tensor([j for j in Y_golds[i] if j != model.padidx])
            for tag in path:
                TP[tag] += ((path == tag) & (gold == tag)).sum()
                FP[tag] += ((path == tag) & (gold != tag)).sum()
                FN[tag] += ((path != tag) & (gold == tag)).sum()
                class_counts[tag] += (gold == tag).sum()
              

100%|█████████████████████████████████████████████| 4/4 [00:05<00:00,  1.46s/it]


In [16]:
precision = TP / (TP + FP)
# avoid nan
nan_mask = torch.isnan(precision)
precision[nan_mask] = 0.

recall = TP / (TP + FN)
# avoid nan
nan_mask = torch.isnan(recall)
recall[nan_mask] = 0.

f1_score = 2 * (precision * recall) / (precision + recall)
# avoid nan
nan_mask = torch.isnan(f1_score)
f1_score[nan_mask] = 0.
# Calculate class weights
class_weights = class_counts / class_counts.sum()

In [17]:
for tag in range(num_tags):
    print(model.tags_vocab.rev_lookup(tag), f1_score[tag])

<unk> tensor(0.)
<pad> tensor(0.)
B_CL tensor(0.9270)
I_V tensor(0.8889)
I_ADV tensor(0.2275)
B_V tensor(0.8962)
B_P tensor(0.9381)
B_A tensor(0.7716)
B_D tensor(0.9347)
B_N tensor(0.8759)
B_PONCT tensor(0.9828)
B_C tensor(0.9336)
B__ tensor(0.9997)
B_ADV tensor(0.3598)
I_N tensor(0.4318)
I_C tensor(0.4000)
I_CL tensor(0.9091)
I_D tensor(0.7143)
I_P tensor(0.3692)
I_PONCT tensor(0.8235)
I_A tensor(0.4545)
B_PREF tensor(0.8000)
B_I tensor(0.)
B_ET tensor(0.)
I_ET tensor(0.)
B_NC tensor(0.)
B_S tensor(0.)
B_X tensor(0.)


In [11]:
weighted_f1_score = (f1_score * class_weights)
weighted_recall = (recall * class_weights)
weighted_precision = (precision * class_weights)

In [12]:
print(sum(weighted_f1_score))
print(sum(weighted_recall))
print(sum(weighted_recall))

tensor(0.9072)
tensor(0.9160)
tensor(0.9160)


tensor(0.5314)
tensor(0.5052)
tensor(0.5134)
